# 第5章：探索的データ解析（Ⅱ）

## 5.1 エリア情報を確認する

### 5.1.1 基本処理の再実行

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df1 = pd.read_csv("../input/gi_train_mm10.csv") # 10月のデータ
df2 = pd.read_csv("../input/gi_train_mm11.csv") # 11月のデータ

df_tmp = pd.concat([df1, df2])
df = df_tmp[df_tmp["customer_id"].notna()].copy()

df_et1 = df.loc[(df["event_type"] == 1)]
df_et2 = df.loc[(df["event_type"] == 2)]
df_et3 = df.loc[(df["event_type"] == 3)]

### 5.1.2 エリア滞在時間の要約統計量の確認

In [ ]:
df_et2["time_duration"].describe()

### 5.1.3 エリア滞在時間のヒストグラムの作成

In [ ]:
bin_min = int(df_et2["time_duration"].min())
bin_max = int(df_et2["time_duration"].max())
edges = range(bin_min, bin_max, 5)
df_et2["time_duration"].hist(bins=edges)

### 5.1.4 「気付き」の確認

In [ ]:
df_et2["area_id"].value_counts()

### 5.1.5 エリアIDの型変換

In [ ]:
df_et2["area_id"] = df_et2["area_id"].astype(int).astype(str)
df_et2["area_id"].dtypes

### 5.1.6 各エリアの要約統計量の確認

In [ ]:
df_et2_byArea = df_et2.groupby("area_id")
df_et2_byArea["time_duration"].describe()

## 5.2 フレームとエリアの特徴を確認する

### 5.2.1 フレームイン/アウトは必ずどのショッパーも1回行う

In [ ]:
print("■フレームインの回数")
print(df_et1["in_time"].notnull().sum())
print("■フレームアウトの回数")
print(df_et1["out_time"].notnull().sum())
print("■ユニークなショッパーの人数")
print(df_et1["customer_id"].nunique())
print("■customer_idに重複がないことの確認")
print(df_et1["customer_id"].duplicated().sum())

### 5.2.2 エリアイン/アウトがないショッパーもいる

In [ ]:
# フレームイン/アウトのレコードを持っているショッパー
list_frame = df_et1["customer_id"].drop_duplicates()
list_frame

In [ ]:
# エリアイン/アウトのレコードを持っているショッパー
list_area = df_et2["customer_id"].drop_duplicates()
list_area

In [ ]:
list_frame_only = list_frame[~list_frame.isin(list_area)]
list_frame_only

In [ ]:
df[df["customer_id"] == "20201026-010001"]

In [ ]:
df[df["customer_id"] == "20201026-010004"]

### 5.2.3 エリアインしなくても商品に接触するショッパーがいる

In [ ]:
# 商品接触のレコードを持っているショッパー
list_contact = df_et3["customer_id"].drop_duplicates()
list_contact

In [ ]:
list_contact_only = list_contact[~list_contact.isin(list_area)]
list_contact_only

In [ ]:
df[df["customer_id"] == "20201026-020247"]

### 5.2.4 1つのエリアにつき複数回イン/アウトをするショッパーがいる

In [ ]:
import collections
count = collections.Counter(df_et2["customer_id"])
count

In [ ]:
df[df["customer_id"] == "20201026-050096"]

## 5.3 性別・年代を確認する準備

In [ ]:
df_customer_attribute = df_et1[["customer_id", "gender", "age", "age_category"]].drop_duplicates()
df_customer_attribute.head(5)

In [ ]:
print("■df_customer_attributeのレコード数")
print(df_customer_attribute["customer_id"].nunique())

df_customer_attribute_new = df_et1[["customer_id", "gender", "age", "age_category"]]
print("■df_customer_attribute_newのレコード数")
print(df_customer_attribute_new["customer_id"].nunique())

## 5.4 pieで円グラフを作成する

### 5.4.1 立寄者の男女比の確認

In [ ]:
gender_counts = df_customer_attribute["gender"].value_counts()
plt.pie(gender_counts, labels=gender_counts.index, autopct="%.1f%%", startangle=90) 
plt.show()
print(gender_counts)

### 5.4.2 立寄者の年代比の確認

In [ ]:
age_counts = df_customer_attribute["age"].value_counts().sort_index(ascending=False)
plt.pie(age_counts, labels=age_counts.index, autopct="%.1f%%", startangle=90) 
plt.show()
print(age_counts)

## 5.5 接触/購入の回数/人数の一覧表を作成する

### 5.5.2 一覧表の作成

#### 接触回数

In [ ]:
ranking_contact_num = df_et3.groupby("product_name")["num_touch"].sum().astype(int).sort_values(ascending = False)
print(ranking_contact_num)

#### 接触人数

In [ ]:
ranking_contact_nop = df_et3.groupby("product_name")["customer_id"].nunique().sort_values(ascending = False)
print(ranking_contact_nop)

#### 購入回数

In [ ]:
df_et3_buy = df_et3.loc[(df_et3["buy_flag"]==1)]
ranking_buy_num = df_et3_buy.groupby("product_name")["num_touch"].sum().astype(int).sort_values(ascending = False)
print(ranking_buy_num)

#### 購入人数

In [ ]:
df_et3_buy = df_et3[df_et3["buy_flag"]==1]
ranking_buy_nop = df_et3_buy.groupby("product_name")["customer_id"].nunique().sort_values(ascending = False)
ranking_buy_nop

#### 全部まとめる

In [ ]:
df_ranking = pd.concat([ranking_contact_num, ranking_contact_nop, ranking_buy_num, ranking_buy_nop], axis=1)
df_ranking.columns = ["接触回数","接触人数","購入回数","購入人数"]
df_ranking

In [ ]:
pd.set_option("display.max_rows", 150) #150行まで表示可能とした

### 5.5.4 商品名・メーカー名の対比表

In [ ]:
# df_et3からproduct_nameとcompanyがユニークなDataFrameを抽出
df_company_tmp = df_et3[["product_name","company"]][~df_et3.duplicated(subset=["product_name","company"])]
# product_nameをインデックスとする
df_company_tmp = df_company_tmp.rename(index=df_company_tmp["product_name"]).copy() 
# 冗長なproduct_name列を削除
df_company = df_company_tmp.drop("product_name", axis=1) 
df_company

### 5.5.5 一覧表への結合

In [ ]:
#結合する
df_ranking_new = pd.concat([df_ranking, df_company], axis=1, join="inner")
df_ranking_new

In [ ]:
print(df_ranking.shape)
print(df_ranking_new.shape)
print(df_ranking_new.isnull().sum())

## 5.6 CVR（接触人数→購入人数）を算出する

### 5.6.2 CVRの算出

In [ ]:
# 新しく列を追加
df_ranking_new["CVR"] = df_ranking_new["購入人数"] / df_ranking_new["接触人数"]
# CVRが高い順に並び変える
df_ranking_new.sort_values("CVR", ascending=False)

## 5.7 メーカー別に集計する

### 5.7.1 メーカー別の集計

In [ ]:
df_ranking_company = df_ranking_new.groupby("company").sum()
df_ranking_company

### 5.7.2 2通りのCVRを考慮したメーカー別の集計

In [ ]:
df_ranking_company = df_ranking_company.drop("CVR", axis=1)
df_ranking_company["CVR（全体）"] = df_ranking_company["購入人数"] / df_ranking_company["接触人数"]
df_ranking_company["CVR（平均）"] = df_ranking_new.groupby("company").mean()["CVR"]
df_ranking_company
#CVR（全体）が高い順に並び変える
df_ranking_company.sort_values("CVR（全体）", ascending=False)

## 5.8 scatterで散布図を作成する

### 5.8.1 scatterで散布図の作成

In [ ]:
plt.scatter(df_ranking_new["購入人数"], df_ranking_new["接触人数"])

### 5.8.2 corrで相関係数の確認

In [ ]:
df_ranking_new.corr()